In [1]:
import pandas as pd
from datetime import datetime

In [6]:
import pandas as pd

# Define file paths
file_path_spx = 'data_redo/^SPX.csv'
file_path_dowjones = 'data_redo/dowjones.csv'
file_path_gdp = 'data_redo/GDPC1.csv'
file_path_debt = 'data_redo/GFDEBTN.csv'
file_path_m2 = 'data_redo/M2SL.csv'
file_path_cpi = 'data_redo/MEDCPIM158SFRBCLE.csv'
file_path_nasdaq = 'data_redo/NASDAQCOM.csv'
file_path_netfi = 'data_redo/NETFI.csv'
file_path_pce = 'data_redo/PCECTPI.csv'

# Load dataframes
df_spx = pd.read_csv(file_path_spx)
df_dowjones = pd.read_csv(file_path_dowjones)
df_gdp = pd.read_csv(file_path_gdp)
df_debt = pd.read_csv(file_path_debt)
df_m2 = pd.read_csv(file_path_m2)
df_cpi = pd.read_csv(file_path_cpi)
df_nasdaq = pd.read_csv(file_path_nasdaq)
df_netfi = pd.read_csv(file_path_netfi)
df_pce = pd.read_csv(file_path_pce)


In [28]:
# Load dataframes
df_spx = pd.read_csv(file_path_spx)
df_dowjones = pd.read_csv(file_path_dowjones)
df_gdp = pd.read_csv(file_path_gdp)
df_debt = pd.read_csv(file_path_debt)
df_m2 = pd.read_csv(file_path_m2)
df_cpi = pd.read_csv(file_path_cpi)
df_nasdaq = pd.read_csv(file_path_nasdaq)
df_netfi = pd.read_csv(file_path_netfi)
df_pce = pd.read_csv(file_path_pce)

# Convert DATE columns to datetime format
df_spx['DATE'] = pd.to_datetime(df_spx['DATE'], format='%Y-%m-%d')
df_dowjones['DATE'] = pd.to_datetime(df_dowjones['DATE'], format='%m/%d/%y')
df_gdp['DATE'] = pd.to_datetime(df_gdp['DATE'], format='%Y-%m-%d')
df_debt['DATE'] = pd.to_datetime(df_debt['DATE'], format='%Y-%m-%d')
df_m2['DATE'] = pd.to_datetime(df_m2['DATE'], format='%Y-%m-%d')
df_cpi['DATE'] = pd.to_datetime(df_cpi['DATE'], format='%Y-%m-%d')
df_nasdaq['DATE'] = pd.to_datetime(df_nasdaq['DATE'], format='%Y-%m-%d')
df_netfi['DATE'] = pd.to_datetime(df_netfi['DATE'], format='%Y-%m-%d')
df_pce['DATE'] = pd.to_datetime(df_pce['DATE'], format='%Y-%m-%d')

# Rename columns to include the respective names
df_spx.rename(columns={'Open': 'SPX_Open', 'High': 'SPX_High', 'Low': 'SPX_Low', 'Close': 'SPX_Close', 'Adj Close': 'SPX_Adj_Close', 'Volume': 'SPX_Volume'}, inplace=True)
df_dowjones.rename(columns={'Open': 'DowJones_Open', 'High': 'DowJones_High', 'Low': 'DowJones_Low', 'Close': 'DowJones_Close'}, inplace=True)
df_nasdaq.rename(columns={'NASDAQCOM': 'NASDAQ_Close'}, inplace=True)

# Optionally, sort each dataframe by DATE if not already sorted
df_spx = df_spx.sort_values(by='DATE').reset_index(drop=True)
df_dowjones = df_dowjones.sort_values(by='DATE').reset_index(drop=True)
df_gdp = df_gdp.sort_values(by='DATE').reset_index(drop=True)
df_debt = df_debt.sort_values(by='DATE').reset_index(drop=True)
df_m2 = df_m2.sort_values(by='DATE').reset_index(drop=True)
df_cpi = df_cpi.sort_values(by='DATE').reset_index(drop=True)
df_nasdaq = df_nasdaq.sort_values(by='DATE').reset_index(drop=True)
df_netfi = df_netfi.sort_values(by='DATE').reset_index(drop=True)
df_pce = df_pce.sort_values(by='DATE').reset_index(drop=True)

In [30]:
file_path_merged_exchange = 'Merged_Exchange_Rates_Data.csv'
df_merged_exchange = pd.read_csv(file_path_merged_exchange)
df_merged_exchange['DATE'] = pd.to_datetime(df_merged_exchange['DATE'], format='%Y-%m-%d')
df_merged_exchange = df_merged_exchange.sort_values(by='DATE').reset_index(drop=True)
df_merged_exchange

,DATE,DEXUSUK,DEXJPUS,DEXUSEU
0,1971-01-04,2.3938,357.73,NaN
1,1971-01-05,2.3949,357.81,NaN
2,1971-01-06,2.3967,357.86,NaN
3,1971-01-07,2.3963,357.87,NaN
4,1971-01-08,2.3972,357.82,NaN
...,...,...,...,...
13880,2024-03-18,1.2736,149.13,1.0886
13881,2024-03-19,1.2713,150.73,1.0859
13882,2024-03-20,1.2713,151.66,1.0856
13883,2024-03-21,1.267,151.59,1.0861


In [31]:
# Determine the minimum date for the DEXUSEU column
min_date_dexuseu = df_merged_exchange['DATE'][df_merged_exchange['DEXUSEU'].notna()].min()

# Filter rows where the date is less than the minimum date for DEXUSEU
df_merged_exchange_filtered = df_merged_exchange[df_merged_exchange['DATE'] >= min_date_dexuseu]

# Convert DEXJPUS to numeric, coercing errors to NaN
df_merged_exchange_filtered['DEXJPUS'] = pd.to_numeric(df_merged_exchange_filtered['DEXJPUS'], errors='coerce')

# Transform DEXJPUS from JP to US to US to JP (inverse the values)
df_merged_exchange_filtered['DEXJPUS'] = 1 / df_merged_exchange_filtered['DEXJPUS']

# Merge all dataframes including the new merged exchange rates dataframe
dfs = [df_spx, df_dowjones, df_gdp, df_debt, df_m2, df_cpi, df_nasdaq, df_netfi, df_pce, df_merged_exchange_filtered]

df_merged_final = pd.merge(dfs[0], dfs[1], on='DATE', how='outer')
for df in dfs[2:]:
    df_merged_final = pd.merge(df_merged_final, df, on='DATE', how='outer')

# Sort the final merged dataframe by DATE
df_merged_final = df_merged_final.sort_values(by='DATE').reset_index(drop=True)

# Display the merged dataframe


C:\Users\aghab\AppData\Local\Temp\ipykernel_28232\1015277535.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_exchange_filtered['DEXJPUS'] = pd.to_numeric(df_merged_exchange_filtered['DEXJPUS'], errors='coerce')
C:\Users\aghab\AppData\Local\Temp\ipykernel_28232\1015277535.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_exchange_filtered['DEXJPUS'] = 1 / df_merged_exchange_filtered['DEXJPUS']


In [33]:
df_merged_final.to_csv('combined.csv', index=False)

In [32]:
df_merged_final.tail()

,DATE,SPX_Open,SPX_High,SPX_Low,SPX_Close,SPX_Adj_Close,SPX_Volume,Open,High,Low,...,GDPC1,GFDEBTN,M2SL,MEDCPIM158SFRBCLE,NASDAQ_Close,NETFI,PCECTPI,DEXUSUK,DEXJPUS,DEXUSEU
24889,2024-03-25,5219.520020,5229.089844,5216.089844,5218.189941,5218.189941,3.331360e+09,39410.54,39430.17,39296.03,...,NaN,NaN,NaN,NaN,16384.47,NaN,NaN,NaN,NaN,NaN
24890,2024-03-26,5228.850098,5235.160156,5203.419922,5203.580078,5203.580078,3.871790e+09,39338.32,39439.44,39277.19,...,NaN,NaN,NaN,NaN,16315.70,NaN,NaN,NaN,NaN,NaN
24891,2024-03-27,5226.310059,5249.259766,5213.919922,5248.490234,5248.490234,3.850500e+09,39461.98,39769.41,39461.98,...,NaN,NaN,NaN,NaN,16399.52,NaN,NaN,NaN,NaN,NaN
24892,2024-03-28,5248.029785,5264.850098,5245.819824,5254.350098,5254.350098,3.998270e+09,39763.74,39868.59,39717.25,...,NaN,NaN,NaN,NaN,16379.46,NaN,NaN,NaN,NaN,NaN
24893,2024-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.251323,NaN,NaN,NaN,NaN,NaN,NaN
